# 24 Convolution Layer

In [1]:
import tensorflow as tf
import numpy as np
import time
import os
import re

from PIL import Image as PI

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

    LABEL
        0 -> NORMAL (3,319)
        1 -> PNEUMOTHORAX (2,084)
        2 -> PNEUMONIA (4,277)

# hyper parameter

In [2]:
training_epochs = 500
batch_size = 20
image_resize = 128
image_width = image_resize
image_height = image_resize
initial_rate = 1e-5
label_len = 3

home_dir = "D:/Backup_data/jongkeun"

image_file = "/dir_images/190819_newTraining/"
test_image_file = "/dir_images/190309_new_test/"

label_file = "/csv/190819_newTraining.csv"
test_label_file = "/csv/190309_new_test.csv"

# label, test_label

In [3]:
label_dir = home_dir + label_file
test_label_dir = home_dir + test_label_file

# label_dir = os.getcwd() + "/csv/181126.csv"
# test_label_dir = os.getcwd() + "/csv/181126_test.csv"

f_label = open(label_dir, 'r')
f_test_label = open(test_label_dir, 'r')

label_data = f_label.readlines()
test_label_data = f_test_label.readlines()

f_label.close()
f_test_label.close()

label_array = np.array(np.reshape(label_data, [-1, label_len]), dtype=np.int32)
test_label_array = np.array(np.reshape(test_label_data, [-1, label_len]), dtype=np.int32)

print(len(label_array))
print(len(test_label_array))

9182
1018


# image, test_image

In [4]:
image_dir = home_dir + image_file
test_image_dir = home_dir + test_image_file

# image_dir = os.getcwd() + "/dir_images/181126/"
# test_image_dir = os.getcwd() + "/dir_images/181126_test/"

image_list = os.listdir(image_dir)
test_image_list = os.listdir(test_image_dir)

image_list = sorted(image_list, key=lambda x: (int(re.sub('\D','',x)),x))
test_image_list = sorted(test_image_list, key=lambda y: (int(re.sub('\D','',y)),y))

image_array = []
test_image_array = []

for i in range(len(image_list)) :
    image_list[i] = image_dir + image_list[i]
    image_array.append(np.array(PI.open(image_list[i]).resize((image_resize, image_resize)).convert('L')))

for j in range(len(test_image_list)) :
    test_image_list[j] = test_image_dir + test_image_list[j]
    test_image_array.append(np.array(PI.open(test_image_list[j]).resize((image_resize, image_resize)).convert('L')))

image_array = np.reshape(image_array, [-1, image_width*image_height])
test_image_array = np.reshape(test_image_array, [-1, image_width*image_height])

image_array = np.array(np.reshape(image_array, [-1, image_width, image_height, 1]), dtype=np.float32)
test_image_array = np.array(np.reshape(test_image_array, [-1, image_width, image_height, 1]), dtype=np.float32)

print(len(image_array))
print(len(test_image_array))

9182
1018


# feeding value

In [5]:
x = tf.placeholder(dtype=tf.float32, shape=[None, image_width, image_height, 1], name='input_image')
y_ = tf.placeholder(dtype=tf.float32, shape=[None, label_len], name='input_label')
learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')

is_training = tf.placeholder(dtype=tf.bool)

# learning layers

In [6]:
def learning_layer(input_data, size, dropout, is_training) :
    layer = tf.layers.conv2d(input_data, size, [3, 3], padding='SAME')
    layer = tf.layers.max_pooling2d(layer, [2, 2], [2, 2], padding='SAME')
    layer = tf.layers.dropout(layer, dropout, is_training)
    
    return layer

with tf.device('/gpu:0') :
    layer0_1 = tf.layers.conv2d(x, 8, [3,3], padding='SAME')
    layer0_2 = tf.layers.conv2d(layer0_1, 128, [3,3], padding='SAME')
    layer1 = learning_layer(layer0_2, 256, dropout=0.7, is_training=is_training)
    
    layer1_1 = tf.layers.conv2d(layer1, 512, [3,3], padding='SAME')
    layer1_2 = tf.layers.conv2d(layer1_1, 256, [3,3], padding='SAME')
    layer2 = learning_layer(layer1_2, 256, dropout=0.7, is_training=is_training)
    
    layer2_1 = tf.layers.conv2d(layer2, 128, [3,3], padding='SAME')
    layer2_2 = tf.layers.conv2d(layer2_1, 256, [3,3], padding='SAME')
    layer2_3 = tf.layers.conv2d(layer2_2, 512, [3,3], padding='SAME')
    layer2_4 = tf.layers.conv2d(layer2_3, 1024, [3,3], padding='SAME')
    layer2_5 = tf.layers.conv2d(layer2_4, 512, [3,3], padding='SAME')
    layer2_6 = tf.layers.conv2d(layer2_5, 512, [3,3], padding='SAME')
    layer2_7 = tf.layers.conv2d(layer2_6, 256, [3,3], padding='SAME')
    layer3 = learning_layer(layer2_7, 512, dropout=0.7, is_training=is_training)
    
    layer3_1 = tf.layers.conv2d(layer3, 1024, [3,3], padding='SAME')
    layer3_2 = tf.layers.conv2d(layer3_1, 256, [3,3], padding='SAME')
    layer3_3 = tf.layers.conv2d(layer3_2, 512, [3,3], padding='SAME')
    layer3_4 = tf.layers.conv2d(layer3_3, 512, [3,3], padding='SAME')
    layer3_5 = tf.layers.conv2d(layer3_4, 1024, [3,3], padding='SAME')
    layer3_6 = tf.layers.conv2d(layer3_5, 512, [3,3], padding='SAME')
    layer3_7 = tf.layers.conv2d(layer3_6, 256, [3,3], padding='SAME')
    layer3_8 = tf.layers.conv2d(layer3_7, 128, [3,3], padding='SAME')
    layer3_9 = tf.layers.conv2d(layer3_8, 256, [3,3], padding='SAME')
    layer4 = learning_layer(layer3_9, 64, dropout=0.7, is_training=is_training)

W1011 14:44:56.546414  3592 deprecation.py:323] From <ipython-input-6-bf1d30a527c3>:9: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W1011 14:44:56.550405  3592 deprecation.py:506] From C:\Users\whdrm\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1011 14:44:56.822703  3592 deprecation.py:323] From <ipython-input-6-bf1d30a527c3>:3: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W1011 14:44:56.930434  3592 deprecation.py:323] From <ipython-input-6-bf1d30a527c3>:4: 

In [7]:
with tf.device('/gpu:0') :
    fully_layer = tf.contrib.layers.flatten(layer4)
    fully_layer = tf.layers.dense(fully_layer, 1024, activation=tf.nn.relu, name='fully_connected_layer')
    fully_layer = tf.layers.dropout(fully_layer, 0.5, is_training, name='fully_connected_dropout')

    fully_layer2 = tf.layers.dense(fully_layer, 512, activation=tf.nn.relu, name='fully_connected_layer2')
    fully_layer2 = tf.layers.dropout(fully_layer2, 0.5, is_training, name='fully_connected_dropout2')

W1011 14:45:00.599572  3592 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1011 14:45:00.600570  3592 deprecation.py:323] From C:\Users\whdrm\Anaconda3\lib\site-packages\tensorflow\contrib\layers\python\layers\layers.py:1634: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W1011 14:45:00.791059  3592 deprecation.py:323] From <ipython-input-7-35197d077b00>:3: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


In [8]:
logits = tf.layers.dense(fully_layer2, label_len, activation=None)

with tf.name_scope('pred') : 
    pred = tf.nn.softmax(logits)
with tf.name_scope('loss') :
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_))
#         loss = -tf.reduce_sum(y_ * tf.log(pred))

with tf.device('/gpu:0') :
    with tf.name_scope('train') :
        train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))
acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

label_output = tf.argmax(pred, 1)
label_input = tf.argmax(y_, 1)

In [9]:
print(label_output)
print(label_input)

Tensor("ArgMax_2:0", shape=(?,), dtype=int64)
Tensor("ArgMax_3:0", shape=(?,), dtype=int64)


In [10]:
saver = tf.train.Saver()

tf.summary.scalar('scalar_loss', loss)

image_shaped_input = tf.reshape(x, [-1, image_resize, image_resize, 1])
tf.summary.image('input', image_shaped_input)

<tf.Tensor 'input:0' shape=() dtype=string>

In [11]:
config = tf.ConfigProto(log_device_placement = True)
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess :
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess=sess, coord=coord)
    
#     f = open(log_dir+log_file, 'w')
    
#     saver = tf.train.import_meta_graph('meta_data_name')
#     saver.restore(sess, tf.train.latest_checkpoint('saved'))
    
    sess.run(tf.global_variables_initializer())
    
#     for var in tf.trainable_variables() :
#         tf.summary.histogram(var.name, var)

#     merged_summary = tf.summary.merge_all()  
#     train_writer = tf.summary.FileWriter(merge_dir, sess.graph)
    
    total_batch = int(len(image_list) / batch_size)
    correct_list = []
    incorrect_list = []
    cost_list = []
    cost_sum_list = []
    pred_list = []
    
#     sensitivity_acc = 0
#     specificity_acc = 0
    total_acc = 0
    cost_sum = 0
    cost_flag = 0
    rate_ = 1
    
    start_time = time.time()
    
    print("")
    
    for epoch in range(training_epochs) :
        total_cost = 0
        
        if epoch%50 == 49 :
                rate_ *= 0.8
                
        if epoch%10 == 9 :
#             rate_ *= 0.8
            midpoint = int(time.time() - start_time)
        
            print("")
            print("=======================================================================================")
            print('{:03d}:{:02d}:{:02d}'.format(midpoint//3600, (midpoint%3600//60), midpoint%60))
            print("=======================================================================================")
            print("")
        
        batch_index = np.random.choice(len(image_array), total_batch, replace=False)
                
        for i in range(total_batch) :                
            _, _loss = sess.run([train, loss], feed_dict={x: image_array[[batch_index[i]]], y_: label_array[[batch_index[i]]], 
                                                          is_training: True, learning_rate: initial_rate*rate_})    

#             _, _loss, _merge = sess.run([train, loss, merged_summary], feed_dict={x: image_array[[batch_index[i]]], y_: label_array[[batch_index[i]]],
#                                                                                   is_training: True, learning_rate: initial_rate*rate_})
            
#             saver.save(sess, save_dir, global_step=total_batch, write_meta_graph=False)
            
            total_cost += _loss

        avg_cost = total_cost/total_batch
        print('Epoch : ', '%4d' % (epoch + 1), '    Avg. cost = ', '{:.4f}'.format(avg_cost))            
        
#         train_writer.add_summary(_merge, epoch)
               
    print("")
    print("=======================================================================================")
    print("================================     Training done     ================================")
    print("=======================================================================================")
    print("")
    
    for test in range(len(test_image_array)) :
        _acc, _label_input, _label_output, _pred = sess.run([acc, label_input, label_output, pred], 
                             feed_dict={x: test_image_array[[test]], y_: test_label_array[[test]], is_training: False})

        total_acc += _acc
        pred_list.append(list(_pred[0]))
        
        print("n: ", test, "    label_input: ", _label_input, "    label_output: ", _label_output)
        
        if _label_input == _label_output :
            correct_list.append(test)   
        else :
            incorrect_list.append(test)
        
    print("")
#     print("       SENSITIVITY ACC : ", '{:.5f}'.format(sensitivity_acc / 200))
#     print("       SPECIFICITY ACC : ", '{:.5f}'.format(specificity_acc / 100))
    print("          TOTAL ACC    : ", '{:.5f}'.format(total_acc / len(test_image_array)))
    print("")
    print(pred_list)
    
#     for cnt in range(len(correct_list)) :
#         data = correct_list[cnt]
#         f.write(str(data)+"\n")
#         print(data)
    
#     tf.train.write_graph(sess.graph, './', pb_, as_text=False)
#     tf.train.write_graph(sess.graph, './', txt_, as_text=True)
#     train_writer.add_graph(sess.graph)
    
    end_time = int(time.time() - start_time)
    
    print("")
    print("=======================================================================================")
    print('{:03d}:{:02d}:{:02d}'.format(end_time//3600, (end_time%3600//60), end_time%60))
    print("=======================================================================================")
    print("")
    
#     for cnt_incorrect in range(len(incorrect_list)) :
#         print(incorrect_list[cnt_incorrect])
    
#     f.close()
    
    coord.request_stop()
    coord.join(thread)

W1011 14:45:04.161048  3592 deprecation.py:323] From <ipython-input-11-55e5bb2bde1f>:6: start_queue_runners (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W1011 14:45:04.162044  3592 queue_runner_impl.py:471] `tf.train.start_queue_runners()` was called when no queue runners were defined. You can safely remove the call to this deprecated function.



Epoch :     1     Avg. cost =  7.7533
Epoch :     2     Avg. cost =  3.4831
Epoch :     3     Avg. cost =  1.5926
Epoch :     4     Avg. cost =  1.5464
Epoch :     5     Avg. cost =  0.9694
Epoch :     6     Avg. cost =  1.6427
Epoch :     7     Avg. cost =  1.5512
Epoch :     8     Avg. cost =  0.7910
Epoch :     9     Avg. cost =  0.7617

000:12:13

Epoch :    10     Avg. cost =  0.9317
Epoch :    11     Avg. cost =  0.8968
Epoch :    12     Avg. cost =  0.8001
Epoch :    13     Avg. cost =  0.7043
Epoch :    14     Avg. cost =  0.8699
Epoch :    15     Avg. cost =  0.7761
Epoch :    16     Avg. cost =  0.7797
Epoch :    17     Avg. cost =  0.8080
Epoch :    18     Avg. cost =  0.7193
Epoch :    19     Avg. cost =  0.8123

000:25:53

Epoch :    20     Avg. cost =  0.6973
Epoch :    21     Avg. cost =  0.6795
Epoch :    22     Avg. cost =  0.7500
Epoch :    23     Avg. cost =  0.7005
Epoch :    24     Avg. cost =  0.6014
Epoch :    25     Avg. cost =  0.5953
Epoch :    26     Avg. co

Epoch :   148     Avg. cost =  0.1176
Epoch :   149     Avg. cost =  0.1801

003:26:19

Epoch :   150     Avg. cost =  0.1610
Epoch :   151     Avg. cost =  0.1631
Epoch :   152     Avg. cost =  0.1840
Epoch :   153     Avg. cost =  0.1341
Epoch :   154     Avg. cost =  0.1332
Epoch :   155     Avg. cost =  0.1330
Epoch :   156     Avg. cost =  0.1297
Epoch :   157     Avg. cost =  0.1593
Epoch :   158     Avg. cost =  0.1233
Epoch :   159     Avg. cost =  0.1387

003:40:17

Epoch :   160     Avg. cost =  0.1208
Epoch :   161     Avg. cost =  0.1563
Epoch :   162     Avg. cost =  0.1496
Epoch :   163     Avg. cost =  0.1404
Epoch :   164     Avg. cost =  0.0824
Epoch :   165     Avg. cost =  0.1760
Epoch :   166     Avg. cost =  0.1139
Epoch :   167     Avg. cost =  0.1436
Epoch :   168     Avg. cost =  0.1194
Epoch :   169     Avg. cost =  0.0930

003:54:15

Epoch :   170     Avg. cost =  0.1021
Epoch :   171     Avg. cost =  0.1762
Epoch :   172     Avg. cost =  0.0782
Epoch :   173 

Epoch :   290     Avg. cost =  0.0510
Epoch :   291     Avg. cost =  0.1941
Epoch :   292     Avg. cost =  0.0185
Epoch :   293     Avg. cost =  0.0227
Epoch :   294     Avg. cost =  0.0313
Epoch :   295     Avg. cost =  0.0152
Epoch :   296     Avg. cost =  0.0193
Epoch :   297     Avg. cost =  0.0283
Epoch :   298     Avg. cost =  0.0406
Epoch :   299     Avg. cost =  0.0118

006:52:40

Epoch :   300     Avg. cost =  0.0286
Epoch :   301     Avg. cost =  0.0100
Epoch :   302     Avg. cost =  0.0025
Epoch :   303     Avg. cost =  0.0783
Epoch :   304     Avg. cost =  0.1864
Epoch :   305     Avg. cost =  0.0234
Epoch :   306     Avg. cost =  0.0188
Epoch :   307     Avg. cost =  0.0326
Epoch :   308     Avg. cost =  0.0163
Epoch :   309     Avg. cost =  0.0246

007:06:21

Epoch :   310     Avg. cost =  0.0493
Epoch :   311     Avg. cost =  0.0153
Epoch :   312     Avg. cost =  0.0815
Epoch :   313     Avg. cost =  0.0228
Epoch :   314     Avg. cost =  0.0190
Epoch :   315     Avg. cos

Epoch :   437     Avg. cost =  0.0005
Epoch :   438     Avg. cost =  0.0007
Epoch :   439     Avg. cost =  0.0002

010:04:06

Epoch :   440     Avg. cost =  0.0003
Epoch :   441     Avg. cost =  0.0058
Epoch :   442     Avg. cost =  0.0084
Epoch :   443     Avg. cost =  0.0853
Epoch :   444     Avg. cost =  0.0035
Epoch :   445     Avg. cost =  0.0002
Epoch :   446     Avg. cost =  0.0001
Epoch :   447     Avg. cost =  0.0102
Epoch :   448     Avg. cost =  0.0005
Epoch :   449     Avg. cost =  0.0058

010:17:46

Epoch :   450     Avg. cost =  0.0022
Epoch :   451     Avg. cost =  0.0004
Epoch :   452     Avg. cost =  0.0011
Epoch :   453     Avg. cost =  0.0003
Epoch :   454     Avg. cost =  0.0277
Epoch :   455     Avg. cost =  0.0029
Epoch :   456     Avg. cost =  0.0754
Epoch :   457     Avg. cost =  0.0001
Epoch :   458     Avg. cost =  0.0002
Epoch :   459     Avg. cost =  0.0000

010:31:25

Epoch :   460     Avg. cost =  0.0001
Epoch :   461     Avg. cost =  0.0002
Epoch :   462 

...

n:  240     label_input:  [0]     label_output:  [0]
n:  241     label_input:  [0]     label_output:  [0]
n:  242     label_input:  [0]     label_output:  [0]
n:  243     label_input:  [0]     label_output:  [0]
n:  244     label_input:  [0]     label_output:  [0]
n:  245     label_input:  [0]     label_output:  [0]
n:  246     label_input:  [0]     label_output:  [0]
n:  247     label_input:  [0]     label_output:  [1]
n:  248     label_input:  [0]     label_output:  [0]
n:  249     label_input:  [0]     label_output:  [1]
n:  250     label_input:  [0]     label_output:  [0]
n:  251     label_input:  [0]     label_output:  [0]
n:  252     label_input:  [0]     label_output:  [0]
n:  253     label_input:  [0]     label_output:  [1]
n:  254     label_input:  [0]     label_output:  [0]
n:  255     label_input:  [0]     label_output:  [0]
n:  256     label_input:  [0]     label_output:  [0]
n:  257     label_input:  [0]     label_output:  [1]
n:  258     label_input:  [0]     label_output

n:  395     label_input:  [1]     label_output:  [1]
n:  396     label_input:  [1]     label_output:  [1]
n:  397     label_input:  [1]     label_output:  [1]
n:  398     label_input:  [1]     label_output:  [1]
n:  399     label_input:  [1]     label_output:  [1]
n:  400     label_input:  [1]     label_output:  [1]
n:  401     label_input:  [1]     label_output:  [1]
n:  402     label_input:  [1]     label_output:  [1]
n:  403     label_input:  [1]     label_output:  [1]
n:  404     label_input:  [1]     label_output:  [1]
n:  405     label_input:  [1]     label_output:  [1]
n:  406     label_input:  [1]     label_output:  [1]
n:  407     label_input:  [1]     label_output:  [1]
n:  408     label_input:  [1]     label_output:  [1]
n:  409     label_input:  [1]     label_output:  [1]
n:  410     label_input:  [1]     label_output:  [1]
n:  411     label_input:  [1]     label_output:  [1]
n:  412     label_input:  [1]     label_output:  [1]
n:  413     label_input:  [1]     label_output

n:  553     label_input:  [1]     label_output:  [1]
n:  554     label_input:  [1]     label_output:  [2]
n:  555     label_input:  [1]     label_output:  [1]
n:  556     label_input:  [1]     label_output:  [0]
n:  557     label_input:  [1]     label_output:  [1]
n:  558     label_input:  [1]     label_output:  [1]
n:  559     label_input:  [1]     label_output:  [1]
n:  560     label_input:  [1]     label_output:  [1]
n:  561     label_input:  [1]     label_output:  [1]
n:  562     label_input:  [1]     label_output:  [1]
n:  563     label_input:  [1]     label_output:  [1]
n:  564     label_input:  [1]     label_output:  [0]
n:  565     label_input:  [1]     label_output:  [1]
n:  566     label_input:  [1]     label_output:  [0]
n:  567     label_input:  [1]     label_output:  [1]
n:  568     label_input:  [2]     label_output:  [2]
n:  569     label_input:  [2]     label_output:  [2]
n:  570     label_input:  [2]     label_output:  [2]
n:  571     label_input:  [2]     label_output

n:  711     label_input:  [2]     label_output:  [0]
n:  712     label_input:  [2]     label_output:  [2]
n:  713     label_input:  [2]     label_output:  [2]
n:  714     label_input:  [2]     label_output:  [2]
n:  715     label_input:  [2]     label_output:  [2]
n:  716     label_input:  [2]     label_output:  [0]
n:  717     label_input:  [2]     label_output:  [2]
n:  718     label_input:  [2]     label_output:  [2]
n:  719     label_input:  [2]     label_output:  [2]
n:  720     label_input:  [2]     label_output:  [2]
n:  721     label_input:  [2]     label_output:  [2]
n:  722     label_input:  [2]     label_output:  [2]
n:  723     label_input:  [2]     label_output:  [2]
n:  724     label_input:  [2]     label_output:  [2]
n:  725     label_input:  [2]     label_output:  [2]
n:  726     label_input:  [2]     label_output:  [1]
n:  727     label_input:  [2]     label_output:  [2]
n:  728     label_input:  [2]     label_output:  [0]
n:  729     label_input:  [2]     label_output

n:  869     label_input:  [2]     label_output:  [2]
n:  870     label_input:  [2]     label_output:  [2]
n:  871     label_input:  [2]     label_output:  [1]
n:  872     label_input:  [2]     label_output:  [2]
n:  873     label_input:  [2]     label_output:  [2]
n:  874     label_input:  [2]     label_output:  [2]
n:  875     label_input:  [2]     label_output:  [1]
n:  876     label_input:  [2]     label_output:  [0]
n:  877     label_input:  [2]     label_output:  [0]
n:  878     label_input:  [2]     label_output:  [0]
n:  879     label_input:  [2]     label_output:  [2]
n:  880     label_input:  [2]     label_output:  [2]
n:  881     label_input:  [2]     label_output:  [2]
n:  882     label_input:  [2]     label_output:  [2]
n:  883     label_input:  [2]     label_output:  [2]
n:  884     label_input:  [2]     label_output:  [2]
n:  885     label_input:  [2]     label_output:  [2]
n:  886     label_input:  [2]     label_output:  [2]
n:  887     label_input:  [2]     label_output